# Prepare file

In [1]:
import pandas as pd
import numpy as np
import os
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import math
import re
import time
import os, psutil

df = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS 2/DATA/MASTER INDEX FILES/total.csv")

In [2]:
listje = list(df['cik'].unique())

with open("/Users/thomasvanhoorn/Documents/allspciks.txt", 'w') as f:
    for item in listje:
        f.write("%s\n" % item)
        
f.close()

In [3]:
cik_cusip = pd.read_csv('/Users/thomasvanhoorn/Documents/THESIS 2/DATA/CIKS CUSIP/CIK CUSIP LINK.csv')
cik_cusip = cik_cusip.drop_duplicates(subset = ["cik"])
cik_cusip = cik_cusip.rename(columns={'cusip': 'cusip2'})

cik_cusip['cusip2'] = cik_cusip['cusip2'].str[:-1]

cik_cusip

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip2,curcd,cik,costat,sic
0,1013,311096,1996.0,INDL,C,D,STD,ADCT.1,00088630,USD,61478,I,3661
15,1045,311296,1996.0,INDL,C,D,STD,AAL,02376R10,USD,6201,A,4512
41,1075,311296,1996.0,INDL,C,D,STD,PNW,72348410,USD,764622,A,4911
67,1078,311296,1996.0,INDL,C,D,STD,ABT,00282410,USD,1800,A,3845
93,1161,311296,1996.0,INDL,C,D,STD,AMD,00790310,USD,2488,A,3674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22267,260774,311202,2002.0,INDL,C,D,STD,CBRE,12504L10,USD,1138118,A,6500
22307,260778,311202,2002.0,INDL,C,D,STD,WCG,94946T10,USD,1279363,I,6324
22341,260996,311207,2007.0,INDL,C,D,STD,HST1,44199X00,USD,1061937,A,6798
22356,294524,311208,2008.0,INDL,C,D,STD,LYB,N5374510,USD,1489393,A,2820


In [4]:
listje = list(cik_cusip['cusip2'].unique())

with open("/Users/thomasvanhoorn/Documents/allspcusips.txt", 'w') as f:
    for item in listje:
        f.write("%s\n" % item)
        
f.close()

In [5]:
cusip_permno = pd.read_csv('/Users/thomasvanhoorn/Documents/THESIS 2/DATA/ALL CUSIPS/CUSIPS.csv')


cusip_permno =  cusip_permno.drop_duplicates(subset = ["PERMNO"])

cusip_permno



,PERMNO,date,SICCD,NCUSIP,TICKER,CUSIP,HSICMG,HSICIG
0,10078,310196,3570.0,86681010,SUNW,86681020,NaN,NaN
169,10104,310196,7370.0,68389X10,ORCL,68389X10,NaN,NaN
481,10107,310196,7370.0,59491810,MSFT,59491810,NaN,NaN
793,10137,310196,4911.0,01741110,AYP,01736110,NaN,NaN
975,10138,310196,6210.0,74147710,TROW,74144T10,NaN,NaN
...,...,...,...,...,...,...,...,...
202647,93096,301009,NaN,NaN,NaN,25667710,NaN,NaN
202794,93132,301009,NaN,NaN,NaN,34959E10,NaN,NaN
202941,93422,300610,NaN,NaN,NaN,74733V10,NaN,NaN
203071,93429,280510,NaN,NaN,NaN,12503M10,NaN,NaN


In [6]:
cusip_permno = cusip_permno.rename(columns={'CUSIP': 'cusip2'})

cusip_permno = cusip_permno.reset_index(drop=True)

#cusip_permno['cusip2'] = cusip_permno['cusip2'].str[:-1]

cusip_permno

,PERMNO,date,SICCD,NCUSIP,TICKER,cusip2,HSICMG,HSICIG
0,10078,310196,3570.0,86681010,SUNW,86681020,NaN,NaN
1,10104,310196,7370.0,68389X10,ORCL,68389X10,NaN,NaN
2,10107,310196,7370.0,59491810,MSFT,59491810,NaN,NaN
3,10137,310196,4911.0,01741110,AYP,01736110,NaN,NaN
4,10138,310196,6210.0,74147710,TROW,74144T10,NaN,NaN
...,...,...,...,...,...,...,...,...
812,93096,301009,NaN,NaN,NaN,25667710,NaN,NaN
813,93132,301009,NaN,NaN,NaN,34959E10,NaN,NaN
814,93422,300610,NaN,NaN,NaN,74733V10,NaN,NaN
815,93429,280510,NaN,NaN,NaN,12503M10,NaN,NaN


In [7]:
linking_table = pd.merge(cik_cusip, cusip_permno, on=['cusip2'], how='inner')
linking_table

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip2,curcd,cik,costat,sic,PERMNO,date,SICCD,NCUSIP,TICKER,HSICMG,HSICIG
0,1013,311096,1996.0,INDL,C,D,STD,ADCT.1,00088630,USD,61478,I,3661,50906,310196,3660.0,00088610,ADCT,NaN,NaN
1,1045,311296,1996.0,INDL,C,D,STD,AAL,02376R10,USD,6201,A,4512,21020,310196,4511.0,00176510,AMR,NaN,NaN
2,1075,311296,1996.0,INDL,C,D,STD,PNW,72348410,USD,764622,A,4911,27991,310196,4911.0,72348410,PNW,NaN,NaN
3,1078,311296,1996.0,INDL,C,D,STD,ABT,00282410,USD,1800,A,3845,20482,310196,2834.0,00282410,ABT,NaN,NaN
4,1161,311296,1996.0,INDL,C,D,STD,AMD,00790310,USD,2488,A,3674,61241,310196,3573.0,00790310,AMD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,199356,311210,2010.0,INDL,C,D,STD,TRIP,89694520,USD,1526520,A,7370,13168,301111,NaN,NaN,NaN,NaN,NaN
813,260774,311202,2002.0,INDL,C,D,STD,CBRE,12504L10,USD,1138118,A,6500,90199,280504,NaN,NaN,NaN,NaN,NaN
814,260778,311202,2002.0,INDL,C,D,STD,WCG,94946T10,USD,1279363,I,6324,90272,300604,NaN,NaN,NaN,NaN,NaN
815,294524,311208,2008.0,INDL,C,D,STD,LYB,N5374510,USD,1489393,A,2820,12345,300910,NaN,NaN,NaN,NaN,NaN


In [19]:
linking_table2 = linking_table[['gvkey','PERMNO','sic','cik','TICKER','cusip2']]
linking_table2 = linking_table2.rename(columns={'cusip2': 'cusip'})
linking_table2.to_csv('/Volumes/USB THOMAS/THESIS NEW/DATA/2. PREPARE MASTERFILE AND FACTOR FILE/linking table.csv')
linking_table2

,gvkey,PERMNO,sic,cik,TICKER,cusip
0,1013,50906,3661,61478,ADCT,00088630
1,1045,21020,4512,6201,AMR,02376R10
2,1075,27991,4911,764622,PNW,72348410
3,1078,20482,3845,1800,ABT,00282410
4,1161,61241,3674,2488,AMD,00790310
...,...,...,...,...,...,...
812,199356,13168,7370,1526520,NaN,89694520
813,260774,90199,6500,1138118,NaN,12504L10
814,260778,90272,6324,1279363,NaN,94946T10
815,294524,12345,2820,1489393,NaN,N5374510


In [57]:
df2 = pd.merge(df, linking_table2, on=['cik'], how='inner')

In [58]:
df2 = df2[['symbol', 'cik','form','access_number','filed_date', 'report_url','filing_url','Y_Q','sic','PERMNO']]




In [59]:
df2

,symbol,cik,form,access_number,filed_date,report_url,filing_url,Y_Q,sic,PERMNO
0,UNP,100885,10-K,0000950109-96-001749,1996-03-26,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR1,4011,48725
1,UNP,100885,10-Q,0000100885-96-000006,1996-05-15,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR2,4011,48725
2,UNP,100885,10-Q,0000100885-96-000008,1996-08-09,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR3,4011,48725
3,UNP,100885,10-Q,0000100885-96-000020,1996-11-13,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR4,4011,48725
4,UNP,100885,10-K,0000100885-97-000002,1997-03-28,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1997.QTR1,4011,48725
...,...,...,...,...,...,...,...,...,...,...
40061,POOL,945841,10-Q,0000945841-21-000141,2021-07-29,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR3,5090,82515
40062,POOL,945841,10-Q,0000945841-21-000178,2021-10-28,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR4,5090,82515
40063,APA,1841666,10-Q,0001673379-21-000012,2021-05-07,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR2,1311,39490
40064,APA,1841666,10-Q,0001871133-21-000007,2021-08-05,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR3,1311,39490


In [60]:
df2['filed_date'] = pd.to_datetime(df2['filed_date'])
df2['date'] = df2['filed_date'].apply(lambda x: x.date())
df2 = df2.rename(columns={'PERMNO': 'permno'})

# IMPORT FACTOR VARS AND ADD TO DF

In [61]:
import pyreadr

result = pyreadr.read_r('/Users/thomasvanhoorn/Documents/Dataset_US.rds')

factor_vars = result[None]

In [62]:
factor_vars['permno'] = factor_vars['permno'].apply(lambda x: int(x))

In [63]:
listje = list(df2['permno'].unique())

listje = np.unique(listje)

factor_vars = factor_vars[factor_vars.permno.isin(listje)]

In [64]:
listje = list(factor_vars['permno'].unique())

listje = np.unique(listje)

df2 = df2[df2.permno.isin(listje)]

In [66]:
df2.to_csv("/Users/thomasvanhoorn/Documents/THESIS 2/DATA/MASTER INDEX FILES/total_incl_permno_final.csv")

# Merge the two files

In [44]:
factor_vars = factor_vars.reset_index(drop=True)

factor_vars['year'] = pd.DatetimeIndex(factor_vars['date']).year
df2['year'] = pd.DatetimeIndex(df2['date']).year


factor_vars['month'] = pd.DatetimeIndex(factor_vars['date']).month
df2['month'] = pd.DatetimeIndex(df2['date']).month


z = df2.groupby('permno')['year'].unique()
z =  pd.DataFrame(z)

z.index.name = 'newhead'
z.reset_index(inplace=True)

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_1455/1027245165.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'] = pd.DatetimeIndex(df2['date']).year
/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_1455/1027245165.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month'] = pd.DatetimeIndex(df2['date']).month


In [46]:
v = factor_vars.loc[factor_vars.permno==z['newhead'][0]][factor_vars.loc[factor_vars.permno==z['newhead'][0]].year.isin(z['year'][0])]

for i in range(1,len(z)):
    y = list(z['year'][i])
    
    v2 = factor_vars.loc[factor_vars.permno==z['newhead'][i]][factor_vars.loc[factor_vars.permno==z['newhead'][i]].year.isin(y)]
    
    v = pd.concat([v,v2], ignore_index = True)
    
factor_vars = v

In [65]:
df2

,symbol,cik,form,access_number,filed_date,report_url,filing_url,Y_Q,sic,permno,date
0,UNP,100885,10-K,0000950109-96-001749,1996-03-26,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR1,4011,48725,1996-03-26
1,UNP,100885,10-Q,0000100885-96-000006,1996-05-15,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR2,4011,48725,1996-05-15
2,UNP,100885,10-Q,0000100885-96-000008,1996-08-09,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR3,4011,48725,1996-08-09
3,UNP,100885,10-Q,0000100885-96-000020,1996-11-13,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR4,4011,48725,1996-11-13
4,UNP,100885,10-K,0000100885-97-000002,1997-03-28,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1997.QTR1,4011,48725,1997-03-28
...,...,...,...,...,...,...,...,...,...,...,...
40061,POOL,945841,10-Q,0000945841-21-000141,2021-07-29,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR3,5090,82515,2021-07-29
40062,POOL,945841,10-Q,0000945841-21-000178,2021-10-28,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR4,5090,82515,2021-10-28
40063,APA,1841666,10-Q,0001673379-21-000012,2021-05-07,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR2,1311,39490,2021-05-07
40064,APA,1841666,10-Q,0001871133-21-000007,2021-08-05,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR3,1311,39490,2021-08-05


In [ ]:
factor_vars.to_csv("/Users/thomasvanhoorn/Documents/THESIS 2/DATA/MASTER INDEX FILES/factor_vars_final.csv")